In [1]:
#!pip install keras-visualizer

In [2]:
%load_ext tensorboard

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from keras_visualizer import visualizer
from IPython.display import Image
import time
import datetime

In [4]:
tf.__version__

'2.14.0'

In [5]:
### ignore TensorFlow INFO messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [6]:
HIDDEN_LAYER_1 = 8
HIDDEN_LAYER_2 = 0

model = tf.keras.Sequential()
model.add(layers.Dense(HIDDEN_LAYER_1, input_dim=2, activation='sigmoid'))

if HIDDEN_LAYER_2 > 0:
    model.add(layers.Dense(HIDDEN_LAYER_2, activation='sigmoid'))

model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 24        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 33 (132.00 Byte)
Trainable params: 33 (132.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), "%d_%d" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [9]:
# dot muss im env/bin Verzeichnis sein, beim Starten des Jupyter Notebooks
visualizer(model, file_format='svg', view=False)
Image(url=f'graph.svg')

In [10]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [11]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [12]:
epochs = 750
if HIDDEN_LAYER_2 > 0:
    epochs = 1500

t1 = time.time()
model.fit(training_data,
          target_data, 
          callbacks=[tensorboard_callback], 
          epochs=epochs)
t2 = time.time()

Epoch 1/750
1/1 [==============================] - 0s 184ms/step - loss: 0.8603 - accuracy: 0.5000
Epoch 2/750
1/1 [==============================] - 0s 11ms/step - loss: 0.7176 - accuracy: 0.5000
Epoch 3/750
1/1 [==============================] - 0s 12ms/step - loss: 0.6957 - accuracy: 0.5000
Epoch 4/750
1/1 [==============================] - 0s 12ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 5/750
1/1 [==============================] - 0s 11ms/step - loss: 0.6929 - accuracy: 0.2500
Epoch 6/750
1/1 [==============================] - 0s 14ms/step - loss: 0.6928 - accuracy: 0.5000
Epoch 7/750
1/1 [==============================] - 0s 12ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 8/750
1/1 [==============================] - 0s 11ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 9/750
1/1 [==============================] - 0s 11ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 10/750
1/1 [==============================] - 0s 12ms/step - loss: 0.6925 - accuracy: 0.5000
Epoch 11/750
1/1 [

In [13]:
print("time %f.3" % (t2 - t1))

time 9.852623.3


In [14]:
model.predict(training_data)

1/1 [==============================] - 0s 36ms/step


array([[0.01676197],
       [0.981627  ],
       [0.9790231 ],
       [0.02222602]], dtype=float32)

## Generate model.cpp

In [15]:
from tinymlgen import port
import os

In [16]:
path = "../pico-tflmicro/examples/xor"

In [17]:
c_code = port(model, optimize=False, pretty_print = True)

open(path + "/model.cpp", "w").write('#include "model.h"\n' + c_code)

INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp3p7otk2m/assets


INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp3p7otk2m/assets
2023-10-21 16:02:01.593451: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-21 16:02:01.593467: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.


12062

<pre>







</pre>
## Save tflite file and code generation

In [18]:
### see https://github.com/eloquentarduino/tinymlgen

import re
import hexdump

def my_port(tflite_model, variable_name='model_data', pretty_print=True):
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    
    preamble = '''
#include "model.h"

// if having troubles with min/max, uncomment the following
// #undef min    
// #undef max
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif
'''
    return preamble + c


In [19]:
def get_filename(optimizer):
    optimezedStr = "_optimized" if optimizer else ""
    return path + "/model_%d_%d%s.tflite" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2, optimezedStr)

In [20]:
def save_tflite(model, optimizer=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimizer:
        optimizers = [tf.lite.Optimize.DEFAULT]        
        converter.optimizations = optimizers
        
        def representative_dataset():
            data = training_data
            yield [data.astype(np.float32)]
        
        converter.representative_dataset = representative_dataset

    filename = get_filename(optimizer)

    tflite_model = converter.convert()
    print("%s %d bytes" % (filename, len(tflite_model)))

    with open(filename, 'wb') as f:
      f.write(tflite_model)
    
    code = my_port(tflite_model)
    open(filename + ".cpp", "w").write(code)

In [21]:
save_tflite(model, optimizer=True)

INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp_qf821nz/assets


INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp_qf821nz/assets


../pico-tflmicro/examples/xor/model_8_0_optimized.tflite 2496 bytes


/Users/d064066/prj/pico-xor/venv/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-10-21 16:02:01.887586: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-21 16:02:01.887601: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


In [22]:
save_tflite(model, optimizer=False)

INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp4vi7zvbu/assets


INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmp4vi7zvbu/assets


../pico-tflmicro/examples/xor/model_8_0.tflite 1892 bytes


2023-10-21 16:02:02.150415: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-21 16:02:02.150427: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.


## Predict tflite files

In [23]:
def predict_tflite(optimizer):
    filename = get_filename(optimizer)
    interpreter = tf.lite.Interpreter(model_path=filename)
    interpreter.allocate_tensors()

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    data = [[tf.cast(0.0, tf.float32), tf.cast(0.0, tf.float32)]]

    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    print("%s %f" % (filename, pred))

    
predict_tflite(optimizer=False)
predict_tflite(optimizer=True)

../pico-tflmicro/examples/xor/model_8_0.tflite 0.016762
../pico-tflmicro/examples/xor/model_8_0_optimized.tflite 0.019531


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/ipykernel_66332/3512523809.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("%s %f" % (filename, pred))
